In [31]:
import pandas as pd

df = pd.read_csv('spx_call_put_100strikes(Worksheet) (2).csv')

print(df.head())
print(df.info())

   Strike             Ticker          Bid          Ask         Last  \
0    5700  SPX 1/15/27 C5700  1555.899902  1561.099854     0.000000   
1    5725  SPX 1/15/27 C5725  1534.099854  1540.800049     0.000000   
2    5750  SPX 1/15/27 C5750  1513.699951  1518.800049     0.000000   
3    5775  SPX 1/15/27 C5775  1492.099854  1498.699951  1435.579956   
4    5800  SPX 1/15/27 C5800  1468.599854  1480.199951  1432.170044   

         IVM  Volm  Strike.1           Ticker.1       Bid.1       Ask.1  \
0  23.571632     0      5700  SPX 1/15/27 P5700  136.500000  137.899994   
1  23.458736     0      5725  SPX 1/15/27 P5725  139.199997  140.800003   
2  23.332682     0      5750  SPX 1/15/27 P5750  142.399994  143.800003   
3  23.224451     0      5775  SPX 1/15/27 P5775  145.199997  146.800003   
4  23.103170     0      5800  SPX 1/15/27 P5800  148.500000  149.899994   

       Last.1      IVM.1  Volm.1  
0  146.250000  23.887341       0  
1  136.600006  23.751657       0  
2  139.729996  23

In [32]:
import sys
import numpy as np
from scipy.optimize import minimize

from riccati import solve_riccati
from rough_heston import rough_heston_mse

In [33]:
# call mid prices
df['Mid_Call'] = (df['Bid'] + df['Ask']) / 2
calls_data = df[['Strike', 'Bid', 'Ask', 'Mid_Call']].copy()

# put mid prices
df['Mid_Put'] = (df['Bid.1'] + df['Ask.1']) / 2
puts_data = df[['Strike.1', 'Bid.1', 'Ask.1', 'Mid_Put']].copy()

# just renaming columns for clarity
calls_data.columns = ['Strike', 'Bid', 'Ask', 'Mid_Call']
puts_data.columns = ['Strike', 'Bid', 'Ask', 'Mid_Put']

calls_data.head()

,Strike,Bid,Ask,Mid_Call
0,5700,1555.899902,1561.099854,1558.499878
1,5725,1534.099854,1540.800049,1537.449951
2,5750,1513.699951,1518.800049,1516.250000
3,5775,1492.099854,1498.699951,1495.399903
4,5800,1468.599854,1480.199951,1474.399903


In [34]:
puts_data.head()

,Strike,Bid,Ask,Mid_Put
0,5700,136.500000,137.899994,137.199997
1,5725,139.199997,140.800003,140.000000
2,5750,142.399994,143.800003,143.099998
3,5775,145.199997,146.800003,146.000000
4,5800,148.500000,149.899994,149.199997


In [35]:
# initial param guess from the paper
initial_guess = [0.6, 1.0, -0.5, 0.05, 0.04, 0.04]

# explicit bounds that we will pass to the optimizer
bounds = [
    (0.51, 0.99),  # alpha (roughness)
    (0.1, 5.0),    # lambda (mean reversion speed)
    (-0.9, 0.7),   # rho (correlation / skew)
    (1e-4, 0.5),   # nu (vol‑of‑vol)
    (1e-4, 0.2),   # theta (level)
    (1e-4, 1.0)    # V0 (initial variance)
]

market_strikes = df['Strike'].values
S0 = 6940.0
r = 0.045
T = 1.0

# helper that prints progress occasionally
def callback(xk):
    if np.random.rand() < 0.01:  # print roughly 1% of the evaluations
        mse = rough_heston_mse(xk, market_strikes, call_market_prices, S0, T, r)
        print("iter", xk, "mse", mse)

# quick sanity check at the starting point
print("initial mse (calls)", rough_heston_mse(initial_guess, market_strikes, calls_data['Mid_Call'].values, S0, T, r, 'call'))
print("initial mse (puts)", rough_heston_mse(initial_guess, market_strikes, puts_data['Mid_Put'].values, S0, T, r, 'put'))

# calls
call_market_prices = calls_data['Mid_Call'].values
result_calls = minimize(
    rough_heston_mse,
    initial_guess,
    args=(market_strikes, call_market_prices, S0, T, r, 'call'),
    method='L-BFGS-B',        # bdd quasi‑Newton
    bounds=bounds,
    options={'ftol':1e-8, 'disp': True, 'maxiter': 100},
    callback=callback
)
print("Calibrated params using calls:", result_calls.x)

# puts
put_market_prices = puts_data['Mid_Put'].values
result_puts = minimize(
    rough_heston_mse,
    initial_guess,
    args=(market_strikes, put_market_prices, S0, T, r, 'put'),
    method='L-BFGS-B',
    bounds=bounds,
    options={'ftol':1e-8, 'disp': True, 'maxiter': 100},
    callback=callback
)
print("Calibrated params using puts:", result_puts.x)


TypeError: rough_heston_mse() takes 6 positional arguments but 7 were given